## Dependances et import des librairies 

In [ ]:
import sys
print(sys.executable)
import plotly.io as pio
pio.renderers
!jupyter labextension list --verbose
#!pip install --upgrade jupyterlab-plotly
#!pip install "plotly>=5" "ipywidgets>=7.6"

In [ ]:
!pip install --upgrade pip
!pip install pandas
#!pip install zstandard
!pip install geopandas
!pip install seaborn
!pip install plotly
!pip install zstandard
!pip install nbformat
!pip install -U kaleido
!pip install scikit-learn

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import os
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import plotly.express as px
from sklearn.cluster import KMeans
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

## Charger les donnees

In [ ]:
df_stations = pd.read_csv("donnees/stations_hb.csv.zst",sep=';',escapechar = '\\')

In [ ]:
f="donnees/donnees_physicochimie.csv.zst"
pc_sample = pd.read_csv(f,nrows=1)
pc_list_cols = pc_sample.columns
pc_list_cat = pc_list_cols[pc_list_cols.str.startswith((
    'Lb','Nom','Mnemo',
    'Cd','Sym','Com'))]

pc_dict_cat = {col: 'category' for col in pc_list_cat}

df_pc = pd.read_csv(
        f,
        sep=',',
        engine='c',
        escapechar='\\',
        dtype=pc_dict_cat,
        parse_dates=[7],
        iterator=False)

# df_hb = pd.read_csv("donnees/donnees_hydrobio.csv.zst",sep=',',escapechar = '\\')


In [ ]:
df_hb = pd.read_csv("donnees/donnees_hydrobio.csv.zst",sep=',',escapechar = '\\')

## Nettoyer les donnees

### Dataframe Hydrobiologique

Pour cela nous allons commencer par nous demander quelles sont les valeurs qui ont des donnees manquantes et commencer a evincer en commencant par elles:

In [ ]:
def pourcentage_valeurs_manquantes(df, colonne):
    """
    Calcule le pourcentage de valeurs manquantes dans une colonne spécifique d'un DataFrame.
    
    Parameters:
    df (pandas.DataFrame): Le DataFrame à analyser
    colonne (str): Le nom de la colonne à vérifier
    
    Returns:
    float: Le pourcentage de valeurs manquantes dans la colonne
    """
    if colonne not in df.columns:
        raise ValueError(f"La colonne '{colonne}' n'existe pas dans le DataFrame.")
    
    total_valeurs = len(df)
    valeurs_manquantes = df[colonne].isnull().sum()
    
    pourcentage = (valeurs_manquantes / total_valeurs) * 100
    
    return round(pourcentage, 2)

In [ ]:
df_with_null_values = df_hb.isnull().any().sort_values().reset_index()
print(df_with_null_values)
df_with_null_values = df_with_null_values.loc[df_with_null_values[0] == True]
df_with_null_values.columns = ['Nom', 'Boolean']

#### Traitement des colonnes avec valeurs manquantes

Ces donnees contiennent des valeurs manquantes:

| Noms colonnes                      | Bool |
|------------------------------------|------|
| DtProdResultatBiologique           | True |
| MnAccredRsIndiceResultatBiologique | True |
| CdPointEauxSurf                    | True |
| CdMethEval                         | True |
| NomProducteur                      | True |
| CdAccredRsIndiceResultatBiologique | True |
| ResIndiceResultatBiologique        | True |

Avant tout, regardons le pourcentage de donnees manquantes pour comprendre l'impact de ces dernieres.

In [ ]:
resultats = []

for colonne in df_with_null_values['Nom']:
    if colonne in df_hb.columns:
        pourcentage = pourcentage_valeurs_manquantes(df_hb, colonne)
        resultats.append({'Colonne': colonne, 'Pourcentage': pourcentage})

df_resultats = pd.DataFrame(resultats)
df_resultats = df_resultats.sort_values('Pourcentage', ascending=False)

fig = px.bar(df_resultats, 
             x='Colonne', 
             y='Pourcentage',
             title='Pourcentage de valeurs manquantes par colonne',
             labels={'Colonne': 'Nom de la colonne', 'Pourcentage': 'Pourcentage de valeurs manquantes'},
             text='Pourcentage')

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(xaxis_tickangle=-45,
                  yaxis_range=[0, 100],  # Le pourcentage c'est entre 0 et 100
                  yaxis_title='Pourcentage de valeurs manquantes (%)',
                  height=600, 
                  margin=dict(b=100))

fig.show(renderer='browser')
#print(df_resultats)

- <ins>DtProdResultatBiologique</ins>: Indication du jour, du mois et de l'année à laquelle le résultat biologique est calculé. **Ce parametre n'a pas d'utilite comme nous utilisons la date de debut des operations biologiques comme referentiel date. Avec plus de 60% de valeurs nulles, elle n'est de toute facon pas exploitable.**
- <ins>MnAccredRsIndiceResultatBiologique</ins> et <ins>CdAccredRsIndiceResultatBiologique</ins>: Information permettant d’indiquer si une analyse a été réalisée dans les conditions d’accréditation. Les valeurs prisent par ce parametre = Inconnu', nan, 'Analyse réalisée sous accréditation', 'Analyse réalisée hors accréditation'.

[//]: # (Tableau des valeurs d'acreditation)

| Code de l'élément | Mnémonique de l'élément | Libellé de l'élément                | Statut de l'élément | Définition de l'élément                                                                                                                                                                                                                                                                                     |
|-------------------|-------------------------|-------------------------------------|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1                 | ACCREDITE               | Analyse réalisée sous accréditation | Validé              | Analyse réalisée par un laboratoire officiellement accrédité pour cette tâche par le Comité Français d'Accréditation (COFRAC) ou un autre organisme d’accréditation similaire, en respectant notamment les spécifications de la norme ISO 17025. L’analyse est fournie sous logo de l’organisme accréditeur |
| 2                 | NON ACCREDITE           | Analyse réalisée hors accréditation | Validé              | Analyse réalisée par un intervenant n'étant pas accrédité pour le paramètre considéré ou analyse réalisée par un intervenant accrédité mais considérant que les conditions de réalisation de l’analyse ne permettent pas la fourniture du résultat sous logo de l’organisme accréditeur.                    |
| 0                 | INCONNU                 | Inconnu                             | Validé              | Analyse réalisée dans des conditions d'accréditation inconnues                                                                                                                                                                                                                                              |


[Accréditation de l'analyse nomenclature 299 (Source du tableau)](https://mdm.sandre.eaufrance.fr/node/297378)

**Dans le cadre de notre analyse, nous allons choisir toutes les lignes, qu'elles soient accredite ou non. Pour ne pas reduire le dataset a une quantite trop limitee de donnees (8428). Cependant, c'est un point a prendre en compte pour des analyses plus justes.**

- <ins>CdPointEauxSurf</ins>: Le point de prélèvement est un sous-espace caractéristique et représentatif pour l'objet qui lui a été défini de la station, qui est clairement identifié et localisé afin d'y effectuer de façon répétitive des mesures pour une connaissance approfondie du milieu à l'endroit de la station. **Valeurs n'ayant aucun impact sur les analyses. Permet d'aider a la reproduction des prelevement pour l'organisme qui le realise.**
- <ins>CdMethEval</ins>: ???? A verifier
- <ins>NomProducteur</ins>: Nom de l’intervenant producteur de l’opération de prélèvement biologique. **Ne permet pas d'aider pour le choix l'analyse.**
- <ins>ResIndiceResultatBiologique</ins>: **Importance majeure car contient les resultats qui servent pour l'analyse. On supprime les 13 lignes avec les valeurs nan.**

Traitement des donnees:

In [ ]:
df_hb['MnAccredRsIndiceResultatBiologique'].unique()
pourcentage_valeurs_manquantes(df_hb, 'MnAccredRsIndiceResultatBiologique')
#df_hb['ResIndiceResultatBiologique'].unique()
df_hb['ResIndiceResultatBiologique'].isnull().sum()

#### Selection des colonnes avec valeurs non manquantes

| Nom |                       Boolean |       |
|----:|------------------------------:|-------|
|   0 |                    Unnamed: 0 | False |
|   1 |    CdStationMesureEauxSurface | False |
|   2 |    LbStationMesureEauxSurface | False |
|   3 |     DateDebutOperationPrelBio | False |
|   4 |                     CdSupport | False |
|   5 |                     LbSupport | False |
|   6 |                  CdProducteur | False |
|   7 | CdParametreResultatBiologique | False |
|   8 |               LbLongParametre | False |
|   9 |           RefOperationPrelBio | False |
|  10 |                 CdUniteMesure | False |
|  11 |                SymUniteMesure | False |
|  12 |  CdRqIndiceResultatBiologique | False |
|  13 |                    MnemoRqAna | False |

- <ins>CdStationMesureEauxSurface</ins>: Permet de lier les donnees avec une station. **Obligatoire**
- <ins>LbStationMesureEauxSurface</ins>: Valeurs qu'on peut retrouver dans la station. **Il n'est donc pas pertinent.**
- <ins>DateDebutOperationPrelBio</ins>: Utilise pour dater les prelevements. **Il est donc necessaire.**
- <ins>CdSupport</ins>: Un support désigne un COMPOSANT DU MILIEU SUR LEQUEL PORTE L’INVESTIGATION, faisant généralement l'objet de prélèvements en vue d'analyses ultérieures, afin d’évaluer sa qualité et celle du milieu. **Necessaire pour l'analyse**
- <ins>CdParametreResultatBiologique</ins>: Ne contient que la valeur **7613**. N'apporte aucune information. **Inutile de garder cette colonne**
- <ins>LbLongParametre</ins>: Nom du parametre teste. Contient qu'une valeur unique "Indice Invertébrés Multimétrique (I2M2)". **N'apporte pas d'information on peut supprimer la colonne**
- <ins>RefOperationPrelBio</ins>: Valeur unique correspondant a un identifiant pour chaque Operation de prelevement. **A garder**
- <ins>CdUniteMesure</ins>: ???? A verifier
- <ins>SymUniteMesure</ins>: ???? A verifier avec CdUniteMesure
- <ins>CdRqIndiceResultatBiologique</ins>: Resultat attendu. Seulement, 13 valeurs ne sont pas utilisables (Code = 0, cf tableau). Il faut les supprimer. **A garder**

  
| Code | Mnémonique          | Libellé                                                                        |
|------|---------------------|--------------------------------------------------------------------------------|
| 0    | Analyse non faite   | Analyse non faite                                                              |
| 1    | Domaine de validité | Résultat > seuil de quantification et < au seuil de saturation ou Résultat = 0 |


- <ins>MnemoRqAna</ins>: Comme la colonne au dessus.

In [ ]:
#df_hb['CdRqIndiceResultatBiologique'].unique()
#pourcentage_valeurs_manquantes(df_hb, 'CdPointEauxSurf')
#df_hb.loc[df_hb['CdRqIndiceResultatBiologique'] == 0].shape
#df_hb.loc[df_hb['MnAccredRsIndiceResultatBiologique'] == 'Analyse réalisée sous accréditation'].count()

### Suppression des colonnes inutiles:
df_hb = df_hb.drop(columns=['DtProdResultatBiologique', 
                            'CdPointEauxSurf', 
                            'NomProducteur', 
                            'LbStationMesureEauxSurface', 
                            'CdParametreResultatBiologique', 
                            'LbLongParametre'])

df_hb = df_hb.dropna(subset='ResIndiceResultatBiologique')


In [ ]:
# Suppression des stations qui n'ont pas de valeurs physicochimiques. Issu du notebook sur les donnees physicochimiques
df_hb_vides = pd.read_csv('donnees/stations_vides.csv')
df_hb_vides

In [ ]:
# Conversion temporaire en chaînes pour effectuer la comparaison
hb_vides_temp = df_hb_vides['CdStationMesureEauxSurface'].astype(str)
hb_temp = df_hb['CdStationMesureEauxSurface'].astype(str)

print(f"Nombre de lignes avant filtration : {len(df_hb)}")

# Filtrage : exclure les valeurs de stations_vides_temp
df_hb = df_hb[~hb_temp.isin(hb_vides_temp)]

# Vérification du résultat
print(f"Nombre de lignes après filtration : {len(df_hb)}")

In [ ]:
df_hb.shape

### Dataframe Stations

In [ ]:
df_with_null_values = df_stations.isnull().any().sort_values().reset_index()
print(df_with_null_values)
df_with_null_values = df_with_null_values.loc[df_with_null_values[0] == True]
df_with_null_values.columns = ['Nom', 'Boolean']

Commencons pas les colonnes qui ont des valeurs manquantes:

In [ ]:
resultats = []

for colonne in df_with_null_values['Nom']:
    if colonne in df_stations.columns:
        pourcentage = pourcentage_valeurs_manquantes(df_stations, colonne)
        resultats.append({'Colonne': colonne, 'Pourcentage': pourcentage})

df_resultats = pd.DataFrame(resultats)
df_resultats = df_resultats.sort_values('Pourcentage', ascending=False)

fig = px.bar(df_resultats, 
             x='Colonne', 
             y='Pourcentage',
             title='Pourcentage de valeurs manquantes par colonne',
             labels={'Colonne': 'Nom de la colonne', 'Pourcentage': 'Pourcentage de valeurs manquantes'},
             text='Pourcentage')

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(xaxis_tickangle=-45,
                  yaxis_range=[0, 100],  # Le pourcentage c'est entre 0 et 100
                  yaxis_title='Pourcentage de valeurs manquantes (%)',
                  height=600, 
                  margin=dict(b=100))

fig.show()
#print(df_resultats)

In [ ]:
#df_stations['PremierMoisAnneeEtiage'].unique()
#df_stations['PremierMoisAnneeEtiage'].loc[df_stations['PremierMoisAnneeEtiage'] == 6].count()
#df_stations = df_stations[df_stations['PremierMoisAnneeEtiage'].isnull()]
df_stations['CodeDepartement'].unique()

- <ins>SuperficieBassinVersantReel</ins>: Cette colonne contient uniquement des valeurs nulles. **A supprimer**
- <ins>PremierMoisAnneeEtiage</ins>:  Le premier mois de l'année d'étiage est le numéro dans l'année civile du premier mois de la période utilisée pour les études statistiques sur les basses eaux. Dans le cas de notre etude nous n'allons pas nous servir de ce parametre pour le moment. **Nous allons simplement supprimer les valeurs non nulles, c'est a dire les lignes qui ont un premier mois different de 12.**
- <ins>SuperficieBassinVersantTopo</ins>: Seulement 259 valeurs sont non nulles. La colonne apporte trop peu d'information pour pouvoir etre utilisable. **A supprimer**
- <ins>ComStationMesureEauxSurface</ins>: Valeurs textes trop fluctuant pour etre analyse. **A supprimer**
- <ins>FinaliteStationMesureEauxSurface</ins>: La finalité de la station constitue le but pour lequel la station de mesure a été créée. Risque de n'avoir aucune importance pour l'analyse mais certaines valeurs comme "Impact d'un rejet domestique" ou "Impact d'un rejet d'élevage" peuvent aider a orienter si certaines stations sont affectees ou non. **Comme elle contient de nombreuses lignes inutiles nous allons la supprimer pour le moment mais elle peut servir plus tard dans l'analyse.**
- <ins>FinaliteStationMesureEauxSurface</ins>: La date d'arrêt d'activité de la station de mesure est la date à laquelle cessent les opérations de prélèvement sur la station de mesure qui ne remplit plus ses fonctions à cause d'événements intervenus sur le réseau hydrographique ; ou bien la date à laquelle le ou les organismes producteurs de données sur la station cessent d'effectuer des prélèvements pour diverses raisons : financières ou autre. **???? A verifier est ce que les donnees de ces stations ne sont de toutes facon pas dans les autres dataset pour les annees que l'on a besoin**
- <ins>DurStationMesureEauxSurface</ins>: Dureté moyenne estimée à dire d'expert à partir de l'ensemble des analyses d'eau connues sur la ou les stations de mesure située(s) sur le tronçon hydrographique. Peut avoir une utilite pour l'analyse de la durete de l'eau sur les valeurs presentes. Cela represente 1296 valeurs. N'a pas d'importance pour l'analyse suivante, mais peutetre plus tard. **A supprimer pour le moment**
- <ins>PkPointTronconEntiteHydroPrincipale</ins>:  La localisation de la station sur le tronçon hydrographique est obtenue à partir du point kilométrique (pk) qui est l'abscisse curviligne de la station le long d'une entité hydrographique par rapport à l'embouchure, mesurée sur la base de sa géométrie dans la BD Carthage et exprimée en kilomètres avec la précision du décamètre. **La mesure n'est pas utile pour l'analyse sachant qu'on a calcule les hydroecoregions.**
- <ins>LibelleNatureStationMesureEauxSurface</ins> et <ins>CodeNatureStationMesureEauxSurface</ins>: Permet de savoir si c'est une mesure manuelle ou automatique. **Ne permet pas d'aider a l'analyse. A supprimer**
- <ins>NomCoursdEau</ins>: Permet de donner le nom du cours d'eau de la station. 20% des lignes n'ont pas de noms. Cette colonne n'est pas utile pour l'analyse mais peut aider a la facilite d'interpretation sur la localisation de la station. **A supprimer sachant qu'elle n'a pas d'importance pour l'analyse, elle peut servir plus tard si besoin pour les visualisations.**.
- <ins>CdTronconHydrographique</ins>:  le tronçon hydrographique est une entité ou partie d'entité située intégralement à l'intérieur d'une zone hydrographique. Fournit une indication plus precise sur la localisation. **N'est pas pertienent pour l'analyse vu qu'on se sert uniquement des Hydroecoregions**
- <ins>CdCoursdEau</ins>: Le code générique est l'identifiant de l'entité hydrographique. Apport une information sur la localisation du cours d'eau sur lequel est present la station. **Comme le parametre precedent, n'apporte pas d'information pour les analyses, sauf si on veut comparer les parametres des stations par cours d'eau, ce qui n'est pas le cas pour le moment**
- <ins>CdBassinDCE</ins>, <ins>NomEuBassinDCE</ins>, <ins>CdEuSsBassinDCEAdmin</ins> et <ins>CdEuBassinDCE</ins>: Un bassin DCE correspond:
    - soit à un district hydrographique national (exemple: Les cours d'eau de la Corse) ;
    - soit à une portion d'un district hydrographique international située sur le territoire d'un état membre (exemples: la Meuse; la Sambre).
  **Les donnees qui donnent des informations sur une localisation autre que l'hydroecoregion n'a pas d'utilite pour notre analyse**
- <ins>NomMasseDEau</ins>, <ins>CdMasseDEau</ins> et <ins>CdEuMasseDEau</ins>: Donne une indication sur la masse d'eau sur laquelle se trouve la station. **Les donnees qui donnent des informations sur une localisation autre que l'hydroecoregion n'a pas d'utilite pour notre analyse**.

de meme pour toutes les autres donnees qui comportent une information sur la localisation. Pour le moment nous allons uniquement faire des analyses sur l'hydroecoregion: 
- <ins>LocPreciseStationMesureEauxSurface, LbRegion, CodeRegion, CodeCommune, LbCommune, LbDepartement, CodeDepartement</ins>

In [ ]:
df_stations = df_stations.drop(columns=['SuperficieBassinVersantReel',
                                       'SuperficieBassinVersantTopo',
                                       'ComStationMesureEauxSurface',
                                       'FinaliteStationMesureEauxSurface',
                                       'DurStationMesureEauxSurface',
                                       'PkPointTronconEntiteHydroPrincipale',
                                       'LibelleNatureStationMesureEauxSurface',
                                       'CodeNatureStationMesureEauxSurface',
                                       'NomCoursdEau',
                                       'CdTronconHydrographique',
                                       'CdCoursdEau',
                                       'CdBassinDCE',
                                       'NomEuBassinDCE',
                                       'CdEuSsBassinDCEAdmin',
                                       'NomMasseDEau',
                                       'CdMasseDEau',
                                       'LocPreciseStationMesureEauxSurface',
                                       'LbRegion',
                                       'CodeRegion',
                                       'CodeCommune',
                                       'LbCommune',
                                       'LbDepartement',
                                       'CodeDepartement',
                                       ])

df_stations = df_stations[df_stations['PremierMoisAnneeEtiage'].isnull()]

Pour toutes les donnees qui n'ont pas de valeurs nulles:

| Index | Nom                                   | Bool  |
|-------|---------------------------------------|-------|
| 0     | CdStationMesureEauxSurface            | False |
| 1     | LbStationMesureEauxSurface            | False |
| 2     | CodeTypEthStationMesureEauxSurface    | False |
| 3     | CoordXStationMesureEauxSurface        | False |
| 4     | CoordYStationMesureEauxSurface        | False |
| 5     | CdProjStationMesureEauxSurface        | False |
| 6     | LibelleProjection                     | False |
| 7     | LibelleTypEthStationMesureEauxSurface | False |
| 8     | AltitudePointCaracteritisque          | False |
| 9     | DateCreationStationMesureEauxSurface  | False |
| 10    | DateMAJInfosStationMesureEauxSurface  | False |

- <ins>CdStationMesureEauxSurface</ins>: Code pour identifier une station. **Obligatoire a garder.**
- <ins>LbStationMesureEauxSurface</ins>: Utile pour faciliter la visualisation des donnees et la localisation de la Station. **A garder**
- <ins>CodeTypEthStationMesureEauxSurface</ins>: Nature (cours d'eau ou plan d'eau) de l'entité hydrographique sur laquelle la station de mesure est localisée. **Pour l'analyse actuelle, n'a pas de sens. A supprimer**
- <ins>CoordXStationMesureEauxSurface</ins> et <ins>CoordYStationMesureEauxSurface</ins>: **Obligatoire pour determiner les hydroecoregions par jointure**
- <ins>CdProjStationMesureEauxSurface</ins>: Uniquement des valeurs 26 ("Lambert 93"). N'apporte pas d'information. **A supprimer**
- <ins>LibelleProjection</ins>: Comme la colonne precedente, ne contient que les valeurs "'RGF93 / Lambert 93'". **A supprimer**
- <ins>LibelleTypEthStationMesureEauxSurface</ins>: Permet de connaitre la position d'une sation suivant deux valeurs: sur un cours d'eau ou sur un plan d'eau. **Pour notre analyse, nous n'allons pas selectionner ces valeurs". **A supprimer**
- <ins>AltitudePointCaracteritisque</ins>: Altitude de la sation. Peut avoir un impact sur les resultats d'analyse. **A garder pour le moment**
- <ins>DateCreationStationMesureEauxSurface</ins>: Date a laquelle la station a ete creee. **Ne permet pas de donner d'indication pour permettre une analyse.**
- <ins>DateMAJInfosStationMesureEauxSurface</ins>: Comme pour la colonne precedente. **A supprimer.**

In [ ]:
#df_stations['AltitudePointCaracteritisque'].unique()

In [ ]:
# Suppression des valeurs qui ne sont pas utiles pour l'analyse.
df_stations = df_stations.drop(columns=['CdProjStationMesureEauxSurface',
                                        'LibelleProjection',
                                        'LibelleTypEthStationMesureEauxSurface',
                                        'DateCreationStationMesureEauxSurface',
                                        'DateMAJInfosStationMesureEauxSurface'
                                       ])

In [ ]:
df_stations.columns

## Afficher les stations sur une carte pour les separer en hydroecoregions

Permet de visuellement se representer les stations dans chaque hydroecoregions et de modifier le dataset pour y ajouter le code et le nom de chaque hydroeco

In [ ]:
# Définir la projection Lambert 93
crs_lambert = 'PROJCS["RGF_1993_Lambert_93",GEOGCS["GCS_RGF_1993",DATUM["D_RGF_1993",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",700000.0],PARAMETER["False_Northing",6600000.0],PARAMETER["Central_Meridian",3.0],PARAMETER["Standard_Parallel_1",49.0],PARAMETER["Standard_Parallel_2",44.0],PARAMETER["Latitude_Of_Origin",46.5],UNIT["Meter",1.0]]'

# Colonnes des coordonnées X et Y
x_col = 'CoordXStationMesureEauxSurface'
y_col = 'CoordYStationMesureEauxSurface'


gdf_stations = gpd.GeoDataFrame(
    df_stations,
    crs=crs_lambert,
    geometry=gpd.GeoSeries(df_stations.apply(lambda row: geom.Point(row[x_col], row[y_col]), axis=1))
)

her = gpd.read_file("./donnees/Hydroecoregions/Hydroecoregion1.shp")

shp_hydroecoregions = her.to_crs(crs_lambert)

#Permet de faire un df en associant une station avec son hydroecoregion
stations_her = gpd.sjoin(gdf_stations, shp_hydroecoregions, predicate='within').to_crs(crs_lambert)

fig, ax = plt.subplots(figsize=(10, 10))

shp_hydroecoregions.plot(ax=ax, color='lightblue', edgecolor='black', alpha=0.5, label='Hydroécorégions')

#station_her.plot(ax=ax, marker='o', color='#ca4553', markersize=0.2, label='Stations de mesure')
stations_her.plot(column='NomHER1',markersize=1, cmap='twilight', legend=True, ax=ax)

#station_her.head()

# Ajouter une légende et un titre
plt.title("Carte des stations de mesure en France avec hydroécorégions et frontières")
plt.legend()

# Afficher la carte
plt.show()

In [ ]:
#stations_her.loc[stations_her['NomHER1'] == 'ALSACE']
#stations_her['NomHER1'].unique()

In [ ]:
#stations_her['CdMasseDEau'].value_counts()
#stations_her['CdMasseDEau'].count()
len(stations_her.columns)
stations_her.columns

In [ ]:
df_hb.head()
df_hb['CdStationMesureEauxSurface'].value_counts().sort_index()
df_hb.dtypes
#df_hb['DateDebutOperationPrelBio'].head()

In [ ]:
# df_hb.loc[df_hb['CdStationMesureEauxSurface'] == 2000990].count()
# cpt_hb_stations = df_hb['LbStationMesureEauxSurface'].value_counts()

# cpt_hb_stations = cpt_hb_stations.reset_index()
# cpt_hb_stations
# fig_cpt = px.line(cpt_hb_stations, x= y='count')
# fig_cpt.show()


## Analyser les colonnes pour les selectionner
Pour selectionner les annees les plus interessantes a utiliser, nous allons ploter le nombre de donnees en fonction des annees. Les annees qui ont le plus de valeurs sont des annees qui permettent d'avoir de meilleures resultats pour les analyses et permettent de reduire le probleme pour 3 ou 4 annees. 

In [ ]:
ser_dates = pd.to_datetime(df_hb['DateDebutOperationPrelBio']).dt.year

ser_dates = ser_dates.value_counts().reset_index()

ser_dates = ser_dates.sort_values('DateDebutOperationPrelBio')

fig = px.bar(ser_dates, x='DateDebutOperationPrelBio', y='count',
             title='Nombre de valeurs par année',
             labels={'Annee': 'Année', 'Nombre': 'Nombre de valeurs'},
             template='plotly_white')

fig.update_xaxes(type='category')
fig.update_layout(xaxis_title='Année',
                  yaxis_title='Nombre de valeurs',
                  bargap=0.2)

fig.show(renderer='browser')

Ensuite on peut choisir de recuperer les valeurs par saison pour pouvoir les comparer entre elles. Nous n'allons que selectionner les annees qui ont le plus de valeurs. C'est a dire des annees plus grandes que 2015. Pour limiter le dataset nous allons nous focus sur les annees entre 2015 et 2018 pour commencer.

In [ ]:
# Convertir la colonne de date en datetime
df_saison = df_hb
df_saison['DateDebutOperationPrelBio'] = pd.to_datetime(df_hb['DateDebutOperationPrelBio'])

# Fonction pour déterminer la saison
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Hiver'
    elif month in [3, 4, 5]:
        return 'Printemps'
    elif month in [6, 7, 8]:
        return 'Ete'
    else:
        return 'Automne'

# Ajouter une colonne 'Saison'
df_saison['Saison'] = df_saison['DateDebutOperationPrelBio'].apply(get_season)
df_saison['Saison']

## Clustering des sations

Le but etant de connaitres les stations en bon etat et celles en mauvais etat. La selection se fait toujours par hydroecorgions

In [ ]:
#df_tmp = df_stations['CdStationMesureEauxSurface'] == 00990
df_stations['CdStationMesureEauxSurface'].unique()

In [ ]:
# Création du masque booléen
masque = df_stations['CdStationMesureEauxSurface'].isin(df_hb['CdStationMesureEauxSurface'])

# Filtrage du DataFrame
resultat = df_stations[masque]
resultat

In [ ]:
# Fonction pour extraire les 6 derniers chiffres
def last_six_digits(x):
    return str(x).zfill(6)[-6:]

# Application de la fonction aux deux DataFrames
df_hb['LastSix'] = df_hb['CdStationMesureEauxSurface'].apply(last_six_digits)
df_stations['LastSix'] = df_stations['CdStationMesureEauxSurface'].apply(last_six_digits)

#Recuperer hydroecoregion 20 (Depots argilo sableux):
stations_her['LastSix'] = stations_her['CdStationMesureEauxSurface'].apply(last_six_digits)
stations_H20 = stations_her[stations_her['CdHER1'] == 20]

# Création du masque pour la comparaison
mask = df_hb['LastSix'].isin(stations_H20['LastSix'])

# Pour recuperer toutes les valeurs uniquement dans l'hydroecoregion n20
df_hb_HER20 = df_hb[mask]
df_hb_HER20 = pd.DataFrame(df_hb_HER20)

df_hb_HER20.nunique()

In [ ]:
df_hb_HER20.shape

In [ ]:
# Supposons que votre DataFrame s'appelle 'df'
# Grouper par le code de la station (les 6 derniers chiffres de la colonne 'code_station')
grouped = df_hb_HER20.groupby(df_hb_HER20['LastSix'].astype(str))

# Compter le nombre de relevés par station
station_counts_HER20 = grouped.size()


# Créer un graphique à barres
# plt.figure(figsize=(12, 6))
# station_counts.plot(kind='bar')
# plt.title('Nombre de relevés par station')
# plt.xlabel('Code de station')
# plt.ylabel('Nombre de relevés')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()


fig = px.bar(station_counts_HER20,
             title='Nombre de releve par station pour l\'hydroecoregion 20',
             labels={'Releve': 'Stations', 'Nombre': 'Nombre de releve'},
             template='plotly_white')

fig.update_layout(xaxis_title='Stations',
                  yaxis_title='Nombre de valeurs',
                  bargap=0.2)

fig.show(renderer='browser')

In [ ]:
df_hb_HER20['ResIndiceResultatBiologique']

In [ ]:
# Fonction pour classifier l'état écologique
def classify_state(value):
    if value > 0.7003:
        return 'Très bon état'
    elif 0.5164 < value <= 0.7003:
        return 'Bon état'
    elif 0.3443 < value <= 0.5164:
        return 'État moyen'
    elif 0.1721 < value <= 0.3443:
        return 'État médiocre'
    else:
        return 'Mauvais état'

# Définir une séquence de couleurs personnalisée
color_sequence = ['#1e8449', '#21618c', '#9c640c', '#943126', '#633974']

# Appliquer la classification
df_hb_HER20['EtatEcologique'] = df_hb_HER20['ResIndiceResultatBiologique'].apply(classify_state)

# Grouper par les 6 derniers chiffres du code station et l'état écologique
grouped = df_hb_HER20.groupby(['LastSix', 'EtatEcologique'])

# Compter le nombre de relevés dans chaque groupe
counts = grouped.size().reset_index(name='Count')

# Créer un graphique à barres empilées avec Plotly
fig = px.bar(counts, 
             x='LastSix', 
             y='Count', 
             color='EtatEcologique',
             title='Distribution des états écologiques par station pour l\'hydroécorégion 20',
             labels={'LastSix': 'Code Station (6 derniers chiffres)', 
                     'Count': 'Nombre de relevés'},
             category_orders={'EtatEcologique': ['Très bon état', 'Bon état', 'État moyen', 'État médiocre', 'Mauvais état']},
             color_discrete_sequence=['#00FF00', '#7FFF00', '#FFFF00', '#FF7F00', '#FF0000'])

# Mise en page
fig.update_layout(barmode='stack', 
                  height=600,
                  xaxis_title='Code Station (6 derniers chiffres)',
                  yaxis_title='Nombre de relevés')

# Rotation des labels de l'axe x pour une meilleure lisibilité
fig.update_xaxes(tickangle=45)

# Afficher le graphique
fig.show(renderer='browser')

# Afficher un résumé des données
summary = counts.groupby('EtatEcologique')['Count'].sum().sort_values(ascending=False)
print("Résumé des états écologiques pour l'hydroécorégion 20:")
print(summary)

# Calculer le pourcentage pour chaque état
total = summary.sum()
percentages = (summary / total * 100).round(2)
print("\nPourcentages des états écologiques:")
print(percentages)


In [ ]:
# Valeurs des proportions pour chacunes des satitions du HER20:
proportions

In [ ]:
proportions_reset = proportions.reset_index()
# Maintenant, fusionnons les DataFrames
df_hb_HER20 = df_hb_HER20.merge(proportions_reset[['LastSix', 'Cluster', 'Qualite']], 
                                 on='LastSix', 
                                 how='left')


In [ ]:
# Vérifiez le résultat
#stations_her['NomHER1'].value_counts().shape

df_hb['Qualite'].unique()

In [ ]:
df_hb_HER20

In [ ]:
# Fonction pour extraire les 6 derniers chiffres
def last_six_digits(x):
    return str(x).zfill(6)[-6:]

# Fonction pour classifier l'état écologique
def classify_state(value, thresholds):
    if value > thresholds[0]:
        return 'Très bon état'
    elif thresholds[1] < value <= thresholds[0]:
        return 'Bon état'
    elif thresholds[2] < value <= thresholds[1]:
        return 'État moyen'
    elif thresholds[3] < value <= thresholds[2]:
        return 'État médiocre'
    else:
        return 'Mauvais état'

# Fonction pour traiter chaque HER (permet de renvoyer un plot pour afficher les donnees)
def process_her(df_hb, stations_her, her_number, thresholds):
    #Desactiver l'erreur a cause de la copie "A value is trying to be set on a copy of a slice from a DataFrame."
    original_setting = pd.options.mode.chained_assignment
    pd.options.mode.chained_assignment = None

    try:
        # Filtrer les stations pour le HER spécifique
        stations_HER = stations_her[stations_her['CdHER1'] == her_number]
        
        # Créer le masque pour la comparaison
        mask = df_hb['LastSix'].isin(stations_HER['LastSix'])
        
        # Filtrer df_hb pour le HER spécifique
        df_hb_HER = df_hb[mask]
        
        # Appliquer la classification
        #df_hb_HER['EtatEcologique'] = df_hb_HER['ResIndiceResultatBiologique'].apply(lambda x: classify_state(x, thresholds))
        # Utilisez :
        df_hb_HER.loc[:, 'EtatEcologique'] = df_hb_HER['ResIndiceResultatBiologique'].apply(lambda x: classify_state(x, thresholds))
        
        # Calculer les proportions pour chaque station
        proportions = df_hb_HER.groupby('LastSix')['EtatEcologique'].value_counts(normalize=True).unstack(fill_value=0)
        
        # Assurer que toutes les colonnes existent
        all_states = ['Très bon état', 'Bon état', 'État moyen', 'État médiocre', 'Mauvais état']
        for state in all_states:
            if state not in proportions.columns:
                proportions[state] = 0
        
        # Appliquer K-means
        kmeans = KMeans(n_clusters=2, random_state=42)
        proportions['Cluster'] = kmeans.fit_predict(proportions[all_states])
        
        # Déterminer quel cluster représente les "bonnes" stations
        cluster_means = proportions.groupby('Cluster')[all_states].mean()
        good_cluster = 0 if (cluster_means.loc[0, 'Très bon état'] + cluster_means.loc[0, 'Bon état']) > (cluster_means.loc[1, 'Très bon état'] + cluster_means.loc[1, 'Bon état']) else 1
        
        # Étiqueter les clusters
        proportions['Qualite'] = proportions['Cluster'].map({good_cluster: 'Plutôt bonne', 1-good_cluster: 'Plutôt mauvaise'})
        
        # Fusionner les résultats avec df_hb_HER
        proportions_reset = proportions.reset_index()
        df_hb_HER = df_hb_HER.merge(proportions_reset[['LastSix', 'Cluster', 'Qualite']], on='LastSix', how='left')
        
        # Visualiser la distribution moyenne des états écologiques par cluster
        cluster_means = proportions.groupby('Qualite')[all_states].mean().reset_index()
        cluster_means_melted = cluster_means.melt(id_vars=['Qualite'], var_name='EtatEcologique', value_name='Proportion')

        fig_dist = px.bar(cluster_means_melted,
                          x='Qualite', 
                          y='Proportion', 
                          color='EtatEcologique',
                          title=f'Distribution moyenne des états écologiques par cluster pour HER {her_number}',
                          labels={'Proportion': 'Proportion moyenne'},
                          color_discrete_sequence=['#00FF00', '#7FFF00', '#FFFF00', '#FF7F00', '#FF0000'])
        fig_dist.update_layout(barmode='stack')

        return df_hb_HER, proportions, fig_dist
        
    finally:
        # Pour reactiver le parametre dans tous les cas.
        pd.options.mode.chained_assignment = original_setting

# Appliquer last_six_digits aux DataFrames principaux
df_hb['LastSix'] = df_hb['CdStationMesureEauxSurface'].apply(last_six_digits)
df_stations['LastSix'] = df_stations['CdStationMesureEauxSurface'].apply(last_six_digits)
stations_her['LastSix'] = stations_her['CdStationMesureEauxSurface'].apply(last_six_digits)

# Dictionnaire des seuils pour chaque HER (uniquement pour le cas general pour le moment)
her_thresholds = {
    2: [0.7078, 0.457, 0.3047, 0.1523],
    7: [0.6916, 0.4362, 0.2908, 0.1454],
    11: [0.7003, 0.5252, 0.3501, 0.1751],
    13: [0.7003, 0.5252, 0.3501, 0.1751],
    15: [0.7003, 0.5164, 0.3443, 0.1721],
    19: [0.7003, 0.5252, 0.3501, 0.1751],
    20: [0.7003, 0.5164, 0.3443, 0.1721],
    9: [0.7003, 0.5164, 0.3443, 0.1721]
}

# Initialiser le dictionnaire pour stocker les résultats
results = {}

# Traiter chaque HER
for her_number, thresholds in her_thresholds.items():
    df_hb_HER, proportions, fig_dist = process_her(df_hb, stations_her, her_number, thresholds)
    results[her_number] = {
        'df': df_hb_HER,
        'proportions': proportions,
        'fig_dist': fig_dist
    }

    all_states = ['Très bon état', 'Bon état', 'État moyen', 'État médiocre', 'Mauvais état']
    
    # Visualisations pour chaque HER
    fig_scatter = px.scatter(proportions.reset_index(), 
                     x='LastSix', 
                     y=['Très bon état', 'Bon état', 'État moyen', 'État médiocre', 'Mauvais état'],
                     color='Qualite',
                     title=f'Clustering des stations par qualité pour HER {her_number}',
                     labels={'LastSix': 'Code Station (6 derniers chiffres)', 
                             'value': 'Proportion', 
                             'variable': 'État écologique'},
                     color_discrete_map={'Plutôt bonne': 'green', 'Plutôt mauvaise': 'red'})
    
    fig_scatter.show(renderer='browser')
    fig_dist.show(renderer='browser')
    
    # Afficher un résumé
    print(f"\nRésumé pour HER {her_number}:")
    print(proportions['Qualite'].value_counts())
    print("\nMoyenne des proportions par cluster:")
    print(proportions.groupby('Qualite')[all_states].mean())

# Vous pouvez maintenant accéder aux résultats pour chaque HER comme ceci:
# df_hb_HER20 = results[20]['df']
# proportions_HER20 = results[20]['proportions']


In [ ]:
proportions.reset_index()

In [ ]:
results[9]['df']

In [ ]:
for dftosave in her_thresholds:
    df_hb_HER = results[dftosave]['df']
    df_hb_HER.to_csv(f'./donnees/resultats_releves_H{dftosave}_station_bonne-mauvaise.csv')

In [ ]:
df_hb_HER20.to_csv('./donnees/resultats_releves_H20_station_bonne-mauvaise.csv')

In [ ]:
filtered_dataset = dataset[~dataset['station_number'].isin(stations_to_exclude['station_number'])]